<a href="https://colab.research.google.com/github/sanAkel/ufs_diurnal_diagnostics/blob/main/GFS/regional_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compare datasets (SST, SSS) in a user specified region

## Install modules


In [ ]:
!pip install cartopy
!pip install copernicusmarine

In [ ]:
import copernicusmarine

import xarray as xr
import numpy as np
import pandas as pd
from datetime import datetime

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

%matplotlib inline

In [ ]:
K_to_C = 273.15 # Kelvin to degC

In [ ]:
def get_cmems_data(dsetID, vNames, lon_start, lon_end, lat_start, lat_end, time_start, time_end):

  data_request = {"dataset_id" : dsetID,
    "longitude" : [lon_start, lon_end],
    "latitude" : [lat_start, lat_end],
    "time" : [time_start, time_end],
    "variables" : vNames}

  cms_data =copernicusmarine.open_dataset(
    dataset_id = data_request["dataset_id"],
    minimum_longitude = data_request["longitude"][0],
    maximum_longitude = data_request["longitude"][1],
    minimum_latitude = data_request["latitude"][0],
    maximum_latitude = data_request["latitude"][1],
    start_datetime = data_request["time"][0],
    end_datetime = data_request["time"][1],
    variables = data_request["variables"])

  return cms_data

In [ ]:
# CMEMS dataset IDs

# OSTIA SST
sst={'dsetID':'METOFFICE-GLO-SST-L4-NRT-OBS-SST-V2',
     'vNames':['analysed_sst'],
     'varName': 'SST'}

# CNS SSS NRT
sss={'dsetID':'cmems_obs-mob_glo_phy-sss_nrt_multi_P1D',
     'vNames':['sos', 'dos'],
     'varName': 'SSS'}

In [ ]:
# Set Copernicus credentials
CMEMS_username, CMEMS_passwd = ["sakella", "HbFPyP9M"]

# set credentials
copernicusmarine.login(username=CMEMS_username, password=CMEMS_passwd)

## Need to figure out coordinates of the region of interest- needs visual inspection!

In [ ]:
# US North East coast
data_date = '2019-12-12'

lon_s, lon_e = [-79, -74]
lat_s, lat_e = [33, 41]


data_type = sst # CMEMS datasets

In [ ]:
full_ds = copernicusmarine.open_dataset(
    dataset_id = data_type["dsetID"],
    variables = data_type["vNames"]
)

# ease comparison with ocean model: unit coversion
full_ds['analysed_sst'] = full_ds['analysed_sst'] - 273.15

In [ ]:
# shading can be important when the data values have a contrast
# https://matplotlib.org/stable/gallery/images_contours_and_fields/pcolormesh_grids.html

fig = plt.figure(figsize=(4, 10))

ax=plt.subplot(111, projection=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines()
ax.add_feature(cfeature.LAND, zorder=10, edgecolor='k')
ax.text(lon_s, lat_e-1, "OSTIA SST\n{}".format(data_date),
        zorder=15, fontsize=12,color='k', transform=ccrs.PlateCarree())

im=full_ds.sel(time=data_date).\
sel(latitude=slice(lat_s, lat_e), longitude=slice(lon_s, lon_e)).\
analysed_sst.plot(ax=ax, transform=ccrs.PlateCarree(),
                  shading='flat', cmap='Set1',
                  add_colorbar=False, )

cbar=plt.colorbar(im, orientation='vertical', shrink=0.5)

ax.set_extent([lon_s, lon_e, lat_s, lat_e])

cbar.ax.set_ylabel('SST [degC]')
ax.set_title('')

gl = ax.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
gl.xlabel_style = {'size': 10}
gl.ylabel_style = {'size': 10}

## User inputs

In [ ]:
data_type = sst # CMEMS dataset to download

# US North East coast
lon_s, lon_e = [-79, -74]
lat_s, lat_e = [33, 41]

date_start = datetime(2020, 2, 1)
fcst_ndays = 10 # how many days from start date?

date_end = datetime(2020, 2, 29) #date_start + pd.Timedelta(days=fcst_ndays)

## Download OSTIA SST dataset for selected region

In [ ]:
ostia_sst = get_cmems_data(data_type["dsetID"], data_type["vNames"],
                           lon_s, lon_e, lat_s, lat_e,
                           date_start.strftime('%Y-%m-%d'),
                           date_end.strftime('%Y-%m-%d'))

## Plot and save dataset for quick later access

In [ ]:
(ostia_sst['analysed_sst']-K_to_C).plot(x="longitude", y="latitude", col="time", col_wrap=10, cmap='Set1')

In [ ]:
ostia_sst.to_netcdf('ostia_sst_cut_out_{}_{}.nc'.format(date_start.strftime('%Y%m%d'), date_end.strftime('%Y%m%d')))